# Scrapping data

En este fichero se realizara la obtención de los datos de concentraciones de contaminantes atmosféricos proporcionados por la estaciones de medición del ayuntamiento de Madrid.

In [1]:
import os
import re
import requests
import urllib.request
from bs4 import BeautifulSoup
from zipfile import ZipFile
import fileinput

In [2]:
# Enlace de la web de datos de calidad del aire horario del ayuntamiento de madrid 
enlace = "https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=f3c0f7d512273410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"

In [3]:
# Se realiza una llamada a la página
url = urllib.request.urlopen(enlace)
CalidadAire=BeautifulSoup(url.read(),'lxml')

In [4]:
# Se genera un diccionario vacío, donde se albergará los años como key y en los enlaces de descarga como values
# Se tendrá que especificar los años que se quieran estudiar
dict_link={}

ano_inicio=2015
ano_fin=2019

In [5]:
# Se busca la etiqueta en la que está alojada la información de los datos de calidad del aire horario, y se
# y se extrae el año y el enlace de descarga de los ficheros
links=CalidadAire.find_all("a", class_="asociada-link ico-zip")
for line in links:
    link='https://datos.madrid.es'+line.get('href')
    ano=int(line.find_previous("p", class_="info-title").string)
    if (ano >= ano_inicio) & (ano <= ano_fin):
        dict_link[ano]=link

Al tener ya un diccionario con los años y los enlaces de descarga de datos horarios para cada año, se procederá a realizar la descarga de los datos para los años especificados anteriormente

In [6]:
#Se crea la carpeta "data" y una subcarpeta "source"
os.makedirs('/Users/Pablo/Documents/TFM/data/source')
# Se establece la carpeta source como la carpeta de trabajo donde se guardarán las descargas
os.chdir('/Users/Pablo/Documents/TFM/data/source')

In [7]:
# Se descargan
for key,value in dict_link.items():
        print("Descargando la información del año %s..."%format(key))
        zipname=str(key)+".zip"
        urllib.request.urlretrieve(value, zipname)
        with ZipFile(zipname, 'r') as zipObj:
            # Get a list of all archived file names from the zip
            listOfFileNames = zipObj.namelist()
            # Iterate over the file names
            for fileName in listOfFileNames:
                # Check filename endswith txt
                if fileName.endswith('.txt'): #donde pone txt antes había csv
                    # Extract a single file from zip
                    zipObj.extract(fileName, './')
        
        print("¡Año %s completado!"%format(key))

Descargando la información del año 2019...
¡Año 2019 completado!
Descargando la información del año 2018...
¡Año 2018 completado!
Descargando la información del año 2017...
¡Año 2017 completado!
Descargando la información del año 2016...
¡Año 2016 completado!
Descargando la información del año 2015...
¡Año 2015 completado!


In [8]:
mes={"ene_mo":"01","ene":"01",
    "feb_mo":"02","feb":"02",
    "mar_mo":"03","mar":"03",
    "abr_mo":"04","abr":"04",
    "may_mo":"05","may":"05",
    "jun_mo":"06","jun":"06",
    "jul_mo":"07","jul":"07",
    "ago_mo":"08","ago":"08",
    "sep_mo":"09","sep":"09",
    "oct_mo":"10","oct":"10",
    "nov_mo":"11","nov":"11",
    "dic_mo":"12","dic":"12"}

In [9]:
# Cambio de nombre de los ficheros que albergan los datos de contaminación del aire, para que todos tengan 
# la estructura yymm.txt

for file in os.listdir():
    #print(file)
    res = re.findall(r'[0-9]+', file)
    #print(res)
    if len(str(res))==8:
        os.remove(file)
        pass
    else:
        if res[0]=='17':
            new_name=res[0]+mes[file[0:3]]+".txt"
            os.rename(file,new_name)
        else:
            new_name=res[0]+mes[file[0:6]]+".txt"
            os.rename(file,new_name)
    

In [10]:
for file in os.listdir():
    if file<'1710.txt':
        for line in fileinput.FileInput(file,inplace=1):
            comas=[2,5,8,10,12,14,16,18,20]
            for i,j in enumerate(comas):
                line=list(line)
                line.insert(i+j,',')
            line=line[0:-1]
            line="".join(line)
            line=re.sub(r"[^V.,\-(0-9)]","N",line)
            line=line.replace("V",",V,")
            line=line.replace("N",",N,")          
            if line[len(line)-1]==',': line=line[0:(len(line)-1)]
            print(line)